Import duckdb to be able to use sql

In [51]:
import duckdb
import pandas as pd
import re

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC;").df()

description

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
5,sakila,staging,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
6,sakila,staging,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",False
7,sakila,staging,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
8,sakila,staging,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
9,sakila,staging,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False


storing data into dictionary with dfs

In [52]:
dfs = {}

with duckdb.connect("data/sakila.duckdb") as conn:
    for name in description["name"]:
        if name[0] != "_":
            dfs[name] = conn.sql(f"FROM staging.{name};").df()

            duckdb.register(name, dfs[name])



Task 1

a)

In [53]:
duckdb.sql("""--sql
    SELECT
           title,
           length
    FROM film
    WHERE
           length >= 180;
""").df().head()

,title,length
0,ALLEY EVOLUTION,180
1,ANALYZE HOOSIERS,181
2,BAKED CLEOPATRA,182
3,CATCH AMISTAD,183
4,CHICAGO NORTH,185


In [ ]:
#using pandas

b)

In [54]:
duckdb.sql("""--sql
    SELECT
           title,
           rating,
           length,
           description
    FROM film
    WHERE (' ' || title || ' ') LIKE '% LOVE %';
""").df()

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
3,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
4,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...


In [55]:
#trial and error
# duckdb.sql("""--sql
#         SELECT 
#             title,
#             rating,
#             length,
#             description
#         FROM film
#         WHERE (' ' || title || ' ') LIKE '% LOVE %';
#         --WHERE title ~ '(^|[^A-Za-z0-9_])love($|[^A-Za-z0-9_])'
#         --WHERE title ~ '(?i)(^| )love( |$)'
#         --WHERE title LIKE 'LOVE %'
#         --    OR title LIKE '% LOVE %' 
#     --    OR title LIKE 'LOVE'
#     --    OR title LIKE '% LOVE';
# """).df()

In [56]:
#using pandas for b
filtered_df = dfs["film"][dfs["film"]['title'].str.contains(r"\blove\b", case=False, na=False)]
result = filtered_df[['title', 'rating', 'length', 'description']]

result


,title,rating,length,description
373,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
447,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
457,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
510,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
534,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...


c)

In [58]:
duckdb.sql("""--sql
    SELECT
            MIN(length) as min_length,
            MAX(length) as max_length,
            ROUND(AVG(length)) as average_length,
            ROUND(MEDIAN(length)) as median_length
    FROM film;
""").df().head()

,min_length,max_length,average_length,median_length
0,46,185,115.0,114.0


In [ ]:
#using pandas



d)

In [69]:
duckdb.sql("""--sql
    SELECT
         -- måste räkna ut vad det kostar per dag genom att dividera kostnad med dagar
            title,
            rental_rate / rental_duration as cost_per_day
    FROM film
    ORDER BY cost_per_day DESC
    LIMIT 10;
""").df()

,title,cost_per_day
0,BACKLASH UNDEFEATED,1.663333
1,BILKO ANONYMOUS,1.663333
2,BEAST HUNCHBACK,1.663333
3,AUTUMN CROW,1.663333
4,ACE GOLDFINGER,1.663333
5,CARIBBEAN LIBERTY,1.663333
6,BEHAVIOR RUNAWAY,1.663333
7,CASPER DRAGONFLY,1.663333
8,AMERICAN CIRCUS,1.663333
9,CASUALTIES ENCINO,1.663333


In [ ]:
#using pandas

e)

In [ ]:
duckdb.sql("""--sql
    SELECT 
            actor.first_name
            actor.last_name
            COUNT(*) as 
    FROM 
""")